# Evaluate models

In [ ]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
# Views 3
from viewser.operations import fetch
from viewser import Queryset, Column
import views_runs
from views_partitioning import data_partitioner, legacy
from stepshift import views
import views_dataviz
from views_runs import storage, ModelMetadata
from views_runs.storage import store, retrieve, fetch_metadata
from views_forecasts.extensions import *
# Other packages
import pickle as pkl
import os

# sklearn

from sklearn.metrics import mean_squared_error

In [ ]:
# Common parameters:

dev_id = 'Fatalities002'
run_id = 'Fatalities002'
EndOfHistory = 508
RunGeneticAlgo = False
level = 'cm'

username = os.getlogin()

steps = [*range(1, 36+1, 1)] # Which steps to train and predict for

fi_steps = [1,3,6,12,36]
# Specifying partitions

calib_partitioner_dict = {"train":(121,408),"predict":(409,456)}
test_partitioner_dict = {"train":(121,456),"predict":(457,504)}
future_partitioner_dict = {"train":(121,504),"predict":(505,516)}
calib_partitioner =  views_runs.DataPartitioner({"calib":calib_partitioner_dict})
test_partitioner =  views_runs.DataPartitioner({"test":test_partitioner_dict})
future_partitioner =  views_runs.DataPartitioner({"future":future_partitioner_dict})

Mydropbox = f'/Users/{username}/Dropbox (ViEWS)/ViEWS/'
localpath = f'/Users/{username}/Pickles/'
overleafpath = f'/Users/{username}/Dropbox (ViEWS)/Apps/Overleaf/ViEWS predicting fatalities/Tables/'



print('Dropbox path set to',Mydropbox)
print('Overleaf path set to',overleafpath)
print('Local path set to',localpath)

In [ ]:
def FatalitiesHistory(Fatality_cutoff=10,Time_cutoff=12):
    ''' Function to retrieve from viewser a dataframe with dependent variable and a characterisation of recent conflict history  '''

    # initialise QS
    history_colname = "ts_ged_sb_f" + str(Fatality_cutoff) + "_t" + str(Time_cutoff)

    queryset = Queryset("fatalities_history", "country_month")
        # target variable
    queryset = queryset.with_column(Column("ln_ged_sb_dep", from_table = "ged2_cm", from_column = "ged_sb_best_sum_nokgi")
                     .transform.ops.ln()
                     .transform.missing.fill()
                    )   
    queryset = queryset.with_column(Column(history_colname, from_table = "ged2_cm", from_column = "ged_sb_best_sum_nokgi")             
                 .transform.missing.replace_na()
                 .transform.bool.gte(Fatality_cutoff)
                 .transform.temporal.time_since()
                 .transform.missing.replace_na()
                 .transform.bool.gte(Time_cutoff)
                                   )
    history_df = queryset.publish().fetch()
    return history_df, history_colname


def CreateEvaluationDf(stored_modelname_test,Fatality_cutoff=10,Time_cutoff=12,NumberOfMonths=48):
    test_df = pd.DataFrame.forecasts.read_store(stored_modelname_test, run=run_id)
    test_df.replace([np.inf, -np.inf], 0, inplace=True) 
    history_df,history_colname = FatalitiesHistory(Fatality_cutoff,Time_cutoff)
    # NOTE: assumption is that panels are balanced!
    NumberOfMonths = 48
    for step in steps:
        colname = history_colname + '_s' + str(step)
        fromdate = test_partitioner_dict['predict'][0] - step
        todate = test_partitioner_dict['predict'][1] - step
        test_df[colname] = history_df[history_colname].loc[fromdate:todate]
    return test_df,history_colname

In [ ]:
EnsembleList = []
genetic = {
        'modelname': 'ensemble_genetic',
        'algorithm': '',
        'depvar': "ln_ged_sb_dep",
        'predstore_calib': 'cm_ensemble_genetic_calib',
        'predstore_test': 'cm_ensemble_genetic_test'   
    }    

EnsembleList.append(genetic)

In [ ]:
for model in EnsembleList:
    stored_modelname_test = model['predstore_test']
    ensemble_test_df,history_colname = CreateEvaluationDf(stored_modelname_test,Fatality_cutoff=10,Time_cutoff=12,NumberOfMonths=48)

In [ ]:
def EvaluateModel(df,colname):
#    print(df[colname].value_counts())
    grouped = df.groupby(colname)

    Evaluation_results = [] # list to hold evaluation results

    stepcols = ['ln_ged_sb_dep']
    for step in steps:
        col = 'step_pred_' + str(step)
        mse_test_all = mean_squared_error(df[col], df['ln_ged_sb_dep'])
        colname = history_colname + '_s' + str(step)
#        print(df[colname].value_counts())
        grouped_dfs = df.groupby(colname)
        percentiles = (0.25,0.5,0.75,0.90,0.95,0.98,0.99,0.995)
        for name, group in grouped_dfs:
            if name == 1:
                mse_test_lowconflict = mean_squared_error(group[col], group['ln_ged_sb_dep'])
    #            print('0', name, mse_lowconflict)
            if name == 0:
                mse_test_highconflict = mean_squared_error(group[col], group['ln_ged_sb_dep'])
    #            print(name)
    #    print(col, mse_test_all, mse_lowconflict, mse_highconflict)
        Results = {
            'MSE_all':  mse_test_all,
            'RMSE_all': np.sqrt(mse_test_all),
            'MSE_lowconflict':  mse_test_lowconflict,
            'RMSE_lowconflict': np.sqrt(mse_test_lowconflict),
            'MSE_highconflict':  mse_test_highconflict,
            'RMSE_highconflict': np.sqrt(mse_test_highconflict),
        }
        Evaluation_results.append(Results)

    Evaluation_results_df = pd.DataFrame(Evaluation_results)
    return Evaluation_results_df

colname = history_colname + '_s3'
df = ensemble_test_df
Evaluation_ensemble_df = EvaluateModel(df,colname)


In [ ]:
colname = history_colname + '_s3'
print(ensemble_test_df[colname].value_counts())
grouped = ensemble_test_df.groupby(colname)
percentiles = (0.25,0.5,0.75,0.90,0.95,0.98,0.99,0.995)
for name, group in grouped:
    print(name)
    print(group['ln_ged_sb_dep'].describe(percentiles = percentiles))

In [ ]:
Evaluation_ensemble_df

In [ ]:
from ModelDefinitions import DefineEnsembleModels

ModelList = DefineEnsembleModels(level)
evaluation_allmodels = Evaluation_ensemble_df[['MSE_all','MSE_lowconflict','MSE_highconflict']]
    
i = 0
for model in ModelList:
    print(i, model['modelname'], model['data_train'])
    stored_modelname_test = model['predstore_test']
    model['test_df'],model['history_colname'] = CreateEvaluationDf(stored_modelname_test,Fatality_cutoff=10,Time_cutoff=12,NumberOfMonths=48)
    colname = model['history_colname'] + '_s3'
    model['Evaluation_results_df'] = EvaluateModel(model['test_df'],colname)
    cn_all = 'MSE_all_' + model['modelname']
    evaluation_allmodels[cn_all] = model['Evaluation_results_df']['MSE_all']
    cn_lc = 'MSE_lowconflict_' + model['modelname']
    evaluation_allmodels[cn_lc] = model['Evaluation_results_df']['MSE_lowconflict']
    cn_hc = 'MSE_highconflict_' + model['modelname']
    evaluation_allmodels[cn_hc] = model['Evaluation_results_df']['MSE_highconflict']
    i = i + 1

In [ ]:
evaluation_allmodels

In [ ]:
ColsToShow = ['MSE_all','MSE_lowconflict',
              'MSE_all_fatalities002_topics_rf','MSE_lowconflict_fatalities002_topics_rf',
              'MSE_all_fatalities002_topics_hurdle_lgb','MSE_lowconflict_fatalities002_topics_hurdle_lgb',
             'MSE_all_fat_topics_rf','MSE_lowconflict_fat_topics_rf',]

evaluation_allmodels[ColsToShow]

In [ ]:
from ModelDefinitions import DefineEnsembleModels

ModelList = DefineEnsembleModels(level)
    
i = 0
for model in ModelList:
    print(i, model['modelname'], model['data_train'])
    i = i + 1
ModelList[8]['Evaluation_results_df']